In [63]:
import os
import json
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, VectorDBQA
from langchain.chains import RetrievalQA
from langchain.document_loaders import DataFrameLoader
import magic
import os
import nltk
import pandas as pd

from IPython.display import Markdown
from IPython.display import display_markdown

In [7]:
# Read and parse the JSON settings file
with open('../settings.json') as settings_file:
    settings = json.load(settings_file)

In [8]:
# Set your OpenAI API key
os.environ['OPENAI_API_KEY'] = settings['OPENAI_API_KEY']

In [9]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/niko/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [12]:
df = pd.read_csv('../data/exports/article_abstracts.csv')

In [13]:
df.head()

,filename,title,abstract,abstract_modified
0,2011 Miller et al - Pea green manure managemen...,Pea green manure management affects organic wi...,organic farmers in semiarid montana desire gre...,organic farmers in semiarid montana desire gre...
1,2012 Guntinas et al - Effects of moisture and ...,Effects of moisture and temperature on net soi...,climate change will lead to changes in soil mo...,climate change will lead to changes in soil mo...
2,2017 Zandvakili et al - The Potential of Green...,The Potential of Green Manure Mixtures to Prov...,the objectives of this greenhouse experiment w...,the objectives of this greenhouse experiment w...
3,2002 Benbi and Richter - A critical review of ...,A critical review of some approaches to modell...,a number of approaches have been used to model...,a number of approaches have been used to model...
4,2018 Li et al - Soil organic matter dynamics i...,Soil organic matter dynamics in long-term temp...,our objective was to examine the dynamics of o...,our objective was to examine the dynamics of o...


In [14]:
loader = DataFrameLoader(df, page_content_column="abstract_modified")

In [18]:
documents = loader.load()

In [16]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

In [19]:
texts = text_splitter.split_documents(documents)

In [20]:
embeddings = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])

In [25]:
docsearch = Chroma.from_documents(texts, embeddings)

Using embedded DuckDB without persistence: data will be transient


In [28]:
# expose this index in a retriever interface
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":2})

In [29]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever, return_source_documents=True)

In [54]:
questions = [
    'How to determine the mineralization index?',
    'How does temperature affect mineralization?',
    'How does moisture affect mineralization?'
    ]

In [55]:
results = list(map(lambda x: qa({"query": x}), questions))

In [57]:
# results

In [69]:
Markdown("# Results")
for result in results:
    display(Markdown(f"## {result['query']}"))
    display(Markdown(f"### {result['result']}"))
    display(Markdown("#### References"))
    for source_document in result["source_documents"]:
        display(Markdown(f"- {source_document.metadata['title']}"))
    

## How to determine the mineralization index?

###  The mineralization index is determined by measuring the amount of microbial biomass at the start of the incubation, determining the increase in NH4+ after boiling with 2 M KCl for 2 h, and extracting total soluble N with 0.01 M CaCl2.

#### References

- A comparison of different indices for nitrogen mineralization

- Evaluating Chemical and Physical Indices of Nitrogen Mineralization Capacity with an Unequivocal Reference

...

## How does moisture affect mineralization?

###  Moisture affects mineralization by affecting the rate of net nitrogen mineralization and by potentially modifying the sensitivity of the mineralization rate to changes in temperature and moisture content.

#### References

- Effects of moisture and temperature on net soil nitrogen mineralization: A laboratory study

- Temperature and Moisture Effects on Microbial Biomass and Soil Organic Matter Mineralization

# References
* [4 Ways to Do Question Answering in LangChain](https://towardsdatascience.com/4-ways-of-question-answering-in-langchain-188c6707cc5a)